# Medzy
## Overview
This project aims to develop a machine learning model capable of interpreting doctors’ handwriting on prescriptions. By accurately detecting and translating challenging handwriting, the model will empower patients to read their prescriptions independently, making it easier for them to purchase their medications without confusion if they run out of medicine.

This model is using Hugging Face's [TrOCR](https://huggingface.co/docs/transformers/en/model_doc/trocr) for classifying the handwritings.

## Get PyTorch Device

### DirectML

In [ ]:
import torch
import torch_directml

device = torch_directml.device()

### CUDA (fallback to CPU if none)

In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Set Up Model and Preprocessing

### Initialize IAMDataset Class

In [2]:
from torch.utils.data import Dataset
from PIL import Image

class IAMDataset(Dataset):
    def __init__(self, root_dir, df, processor, max_target_length=128):
        self.root_dir = root_dir
        self.df = df
        self.processor = processor
        self.max_target_length = max_target_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # get file name + text 
        file_name = self.df['IMAGE'][idx]
        text = self.df['MEDICINE_NAME'][idx]

        # prepare image (i.e. resize + normalize)
        image = Image.open(self.root_dir + file_name).convert("RGB")
        pixel_values = self.processor(image, return_tensors="pt").pixel_values

        # add labels (input_ids) by encoding the text
        labels = self.processor.tokenizer(
            text, 
            padding="max_length", 
            max_length=self.max_target_length
        ).input_ids

        # important: make sure that PAD tokens are ignored by the loss function
        labels = [label if label != self.processor.tokenizer.pad_token_id else -100 for label in labels]

        encoding = {"pixel_values": pixel_values.squeeze(), "labels": torch.tensor(labels)}
        return encoding

### Loading Model and Preprocessing

In [4]:
# Load model
from transformers import VisionEncoderDecoderModel

model = VisionEncoderDecoderModel.from_pretrained("./model-output/OCR", local_files_only=True)
model.to(device)
model.eval()

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.49.0"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "relu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_model": 1024,
  "decoder_attention_heads": 16,
  "decode

VisionEncoderDecoderModel(
  (encoder): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=False)
              (key): Linear(in_features=768, out_features=768, bias=False)
              (value): Linear(in_features=768, out_features=768, bias=False)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linea

In [5]:
# Load data
import pandas as pd

test_df = pd.read_csv("./Dataset/Testing/testing_labels.csv", delimiter = ",")

# Preprocess data
from transformers import TrOCRProcessor

processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")

test_dataset = IAMDataset(
    root_dir='./Dataset/Testing/testing_words/',
    df=test_df,
    processor=processor
)

# Creating Data Loader
from torch.utils.data import DataLoader

test_dataloader = DataLoader(test_dataset, batch_size=8)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


### Testing

In [6]:
from tqdm.notebook import tqdm

predicted_labels = []

### Run Inference
with torch.no_grad():
    for batch in tqdm(test_dataloader):
        # run batch generation
        outputs = model.generate(batch["pixel_values"].to(device))

        # Decode predictions
        decoded_predictions = [processor.decode(ids, skip_special_tokens=True) for ids in outputs]

        # Store results
        predicted_labels.extend(decoded_predictions)

  0%|          | 0/98 [00:00<?, ?it/s]

In [7]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Get all unique true labels from the dataset
unique_labels = set(test_df["MEDICINE_NAME"].astype(str))  # Ensure labels are strings

# Add "unknown" to unique labels so sklearn treats it as a valid class
unique_labels.add("unknown")

# Replace unknown predictions with "unknown"
cleaned_predictions = [
    pred if pred in unique_labels else "unknown"
    for pred in predicted_labels
]

# Compute metrics
accuracy = accuracy_score(test_df["MEDICINE_NAME"], cleaned_predictions)
precision, recall, f1, _ = precision_recall_fscore_support(test_df["MEDICINE_NAME"], cleaned_predictions, labels=list(unique_labels), average="macro", zero_division=0)

# Print results
print(f"Model Performance:")
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")
print(f"F1-Score: {f1 * 100:.2f}%")

Model Performance:
Accuracy: 10.90%
Precision: 20.68%
Recall: 10.76%
F1-Score: 12.43%


In [8]:
predicted_labels

['AcAetaeta',
 'AcAetaeta',
 'AcAeta',
 'Aetaetaeta',
 'AcAeta',
 'AcAeta',
 'Aetaetaeta',
 'AcAetaeta',
 'Aetaetaeta',
 'AcB',
 'Ace',
 'Ace',
 'Ace',
 'Ace',
 'Ace',
 'Acecece',
 'Aceacceacacac',
 'Ace',
 'Ace',
 'Ace',
 'AlrolatArolrolrol',
 'atAlrolalAt',
 'atAlrolArolrolrol',
 'atAlrolArolrolrol',
 'AlrolKrolrolrol',
 'AlrolFlrolrolrol',
 'rolAlatArolrolrol',
 'Arolrolrol',
 'Arolrolrol',
 'AlRrolrol',
 'AxAmod',
 'AtAmod',
 'AxAinin',
 'Amod',
 'Amod',
 'A',
 'Amod',
 'Amod',
 'Amod',
 'Amod',
 'AtAzriin',
 'AtAzriininrocinininzininin',
 'AtAtriininzininininrocinin',
 'AtAtriininzininininrocinin',
 'AtAzriinzinininin',
 'AtMetsinininrocininzininin',
 'AtAzriinininzininzinzininin',
 'Atriininzininininrocinin',
 'AtAtriininzinininin',
 'AtAzriinin',
 'AxinodO',
 'AxAxAxinodAinin',
 'AxinodO',
 'AxinodO',
 'AxAzinin',
 'inAxAxAxodAinin',
 'inAxAxAxAininin',
 'AxAxAxodOin',
 'Axinod',
 'AxAxAxOodinodinin',
 'Azithrocinininrocinrocrocin',
 'Azithrocrocrocinrocinininrocroc',
 'Azithroc

In [9]:
import json

with open('output.txt', 'w') as filehandle:
    json.dump(predicted_labels, filehandle)